# Churn Prediction Model Inference

## Inference with the Champion model

With Models in Unity Catalog, they can be loaded for use in batch inference pipelines. The generated predictions can used to devise customer retention strategies, or be used for analytics. The model in use is the __Champion__ model, and we will load this for use in our pipeline.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/mlops-uc-end2end-5.png?raw=true" width="1200">

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1832744760933926&notebook=%2F01-mlops-quickstart%2F05_batch_inference&demo_name=mlops-end2end&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fmlops-end2end%2F01-mlops-quickstart%2F05_batch_inference&version=1">

### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-mlops-end2end-edgar_aguilerarod` from the dropdown menu ([open cluster configuration](https://dbc-07122dbb-1c85.cloud.databricks.com/#setting/clusters/0102-173414-9ev1v92w/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('mlops-end2end')` or re-install the demo: `dbdemos.install('mlops-end2end')`*

In [0]:
%pip install --quiet mlflow==2.14.3
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $setup_inference_data=true

##Deploying the model for batch inferences

<!--img style="float: right; margin-left: 20px" width="600" src="https://github.com/QuentinAmbard/databricks-demo/raw/main/retail/resources/images/churn_batch_inference.gif" /-->

Now that our model is available in the Unity Catalog Model Registry, we can load it to compute our inferences and save them in a table to start building dashboards.

We will use MLFlow function to load a pyspark UDF and distribute our inference in the entire cluster. If the data is small, we can also load the model with plain python and use a pandas Dataframe.

If you don't know how to start, you can get sample code from the __"Artifacts"__ page of the model's experiment run.

## Run inferences

In [0]:
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository

requirements_path = ModelsArtifactRepository(f"models:/{catalog}.{db}.mlops_churn@Champion").download_artifacts(artifact_path="requirements.txt") # download model from remote registry

In [0]:
%pip install --quiet -r $requirements_path
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup

### Batch inference on the Champion model

We are ready to run inference on the Champion model. We will load the model as a Spark UDF and generate predictions for our customer records.

For simplicity, we assume that features have been extracted for the new customer records and these are already stored in the feature table. These are typically done by separate feature engineering pipelines.

In [0]:
import mlflow
# Load customer features to be scored
inference_df = spark.read.table(f"mlops_churn_inference")
# Load champion model as a Spark UDF
champion_model = mlflow.pyfunc.spark_udf(spark, model_uri=f"models:/{catalog}.{db}.mlops_churn@Champion")

# Batch score
preds_df = inference_df.withColumn('predictions', champion_model(*champion_model.metadata.get_input_schema().input_names()))

display(preds_df)


That's it! Our data can now be saved as a table and re-used by the Data Analyst / Marketing team to take special action and reduce Churn risk on these customers!

Your data will also be available within Genie to answer any churn-related question using plain text english!

### Conclusion

This is all for the quickstart demo! We have looked at basic concepts of MLOps and how Databricks helps you achieve them. They include:

- Feature engineering and storing feature tables with labels in Databricks
- AutoML, model training and experiment tracking in MLflow
- Registering models as Models in Unity Catalog for governed usage
- Model validation, Champion-Challenger testing, and model promotion
- Batch inference by loading the model as a pySpark UDF

We hope you've enjoyed this demo. As the next step, look out for our Advanced End-to-end MLOps demo, which will include more in-depth walkthroughs on the following aspects of MLOps:

- Feature serving and Feature Store
- Data and model monitoring
- Deployment for real-time inference